In [1]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# Paso 1: Configuración de conexión a PostgreSQL
try:
    conexionfacua=psycopg2.connect(
    database="preciosfacua",
    user="postgres",
    password="admin",
    host="localhost",
    port="5432")
except OperationalError as e:
    if e.pgcode ==errorcodes.INVALID_PASSWORD:
        print("La constraseña es incorrecta")
    elif e.pgcode ==errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
cursorfacua = conexionfacua.cursor()


In [3]:
# Paso 2: Crear las tablas en la base de datos
cursorfacua.execute("""
    CREATE TABLE IF NOT EXISTS supermercados (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(255) UNIQUE NOT NULL
    );
""")

cursorfacua.execute("""
    CREATE TABLE IF NOT EXISTS categoria (
        id SERIAL PRIMARY KEY,
        nombre VARCHAR(255) UNIQUE NOT NULL
    );
""")

cursorfacua.execute("""
    CREATE TABLE IF NOT EXISTS datos (
        id SERIAL PRIMARY KEY,
        fecha DATE NOT NULL,
        precio FLOAT NOT NULL,
        variacion VARCHAR(50),
        nombre VARCHAR(255) NOT NULL,
        supermercado_id INTEGER REFERENCES supermercados(id),
        categoria_id INTEGER REFERENCES categoria(id)
    );
""")

# Confirmar la creación de las tablas
conexionfacua.commit()

In [4]:
# Primero, deshacer la transacción fallida
conexionfacua.rollback()

cursorfacuafacua = conexionfacua.cursor()
cursorfacuafacua

<cursor object at 0x0000028863832A40; closed: 0>

In [15]:
# Paso 3: Cargar los datos desde el archivo CSV
data = pd.read_csv('../data/base/DatospreciosFACUA_2024-10-26.csv')

In [6]:
data

,Día,Precio (€),Variación,Nombre,Supermercado,Tipo producto
0,12/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
1,13/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
2,14/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
3,15/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
4,16/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
...,...,...,...,...,...,...
104782,22/10/2024,"7,74",=,PULEVA Proteína extra pro Leche de vaca desnat...,Alcampo,Leche
104783,23/10/2024,"7,74",=,PULEVA Proteína extra pro Leche de vaca desnat...,Alcampo,Leche
104784,24/10/2024,"7,74",=,PULEVA Proteína extra pro Leche de vaca desnat...,Alcampo,Leche
104785,25/10/2024,"7,74",=,PULEVA Proteína extra pro Leche de vaca desnat...,Alcampo,Leche


In [7]:
# Palabras clave y categorías
palabras_clave = {
    'aceite de girasol': 'Aceite de girasol',
    'aceite de oliva intenso': 'Aceite De Oliva Intenso',
    'aceite de oliva suave': 'Aceite De Oliva Suave',
    'aceite de oliva virgen': 'Aceite De Oliva Virgen',
    'virgen extra': 'Aceite de oliva virgen extra',
    'sin lactosa': 'Leche Sin Lactosa',
}

# Función para asignar categorías basadas en palabras clave
def asignar_categoria(texto):
    for palabra, categoria in palabras_clave.items():
        if palabra in texto.lower():  # Convierte a minúsculas para evitar problemas de mayúsculas/minúsculas
            return categoria
    return 'Leche'  # Asigna 'Otros' si no se encuentra ninguna palabra clave

# Crear la nueva columna 'categoria' usando la función
data['categoria'] = data['Nombre'].apply(asignar_categoria)

In [8]:
data['Tipo producto']=data['categoria']
data=data.drop(columns='categoria')

In [10]:
data

,Día,Precio (€),Variación,Nombre,Supermercado,Tipo producto
0,12/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
1,13/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
2,14/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
3,15/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
4,16/07/2024,"1,45",=,"Aceite de girasol refinado 0,2º Hacendado 1 l.",Mercadona,Aceite de girasol
...,...,...,...,...,...,...
104782,22/10/2024,"7,74",=,PULEVA Proteína extra pro Leche de vaca desnat...,Alcampo,Leche Sin Lactosa
104783,23/10/2024,"7,74",=,PULEVA Proteína extra pro Leche de vaca desnat...,Alcampo,Leche Sin Lactosa
104784,24/10/2024,"7,74",=,PULEVA Proteína extra pro Leche de vaca desnat...,Alcampo,Leche Sin Lactosa
104785,25/10/2024,"7,74",=,PULEVA Proteína extra pro Leche de vaca desnat...,Alcampo,Leche Sin Lactosa


In [11]:
data.dtypes

Día              object
Precio (€)       object
Variación        object
Nombre           object
Supermercado     object
Tipo producto    object
dtype: object

In [12]:
data['Día']= pd.to_datetime(data['Día'], format='%d/%m/%Y')
data['Precio (€)']= data['Precio (€)'].str.replace(',', '.').astype(float)

In [13]:
data.dtypes

Día              datetime64[ns]
Precio (€)              float64
Variación                object
Nombre                   object
Supermercado             object
Tipo producto            object
dtype: object

In [14]:
# Paso 4: Insertar supermercados y categorías únicas en las tablas respectivas
supermercados_unicos = data['Supermercado'].unique()
queryinsercion1 = "INSERT INTO supermercados (nombre) VALUES (%s) ON CONFLICT (nombre) DO NOTHING;"
cursorfacua.executemany(queryinsercion1, [(supermercado,) for supermercado in supermercados_unicos])

categorias_unicas = data['Tipo producto'].unique()
queryinsercion2 = "INSERT INTO categoria (nombre) VALUES (%s) ON CONFLICT (nombre) DO NOTHING;"
cursorfacua.executemany(queryinsercion2, [(categoria,) for categoria in categorias_unicas])

conexionfacua.commit()

# Paso 5: Insertar los datos en la tabla de datos
for _, row in data.iterrows():
    # Convertir la fecha y el precio de cada fila
    fecha = row['Día']
    precio = row['Precio (€)']

    # Obtener los IDs correspondientes de supermercado y categoría
    cursorfacua.execute("SELECT id FROM supermercados WHERE nombre = %s;", [row['Supermercado']])
    supermercado_id = cursorfacua.fetchone()[0]

    cursorfacua.execute("SELECT id FROM categoria WHERE nombre = %s;", [row['Tipo producto']])
    categoria_id = cursorfacua.fetchone()[0]

    # Insertar el registro en la tabla de datos
    cursorfacua.execute("""
        INSERT INTO datos (fecha, precio, variacion, nombre, supermercado_id, categoria_id)
        VALUES (%s, %s, %s, %s, %s, %s);
    """, (fecha, precio, row['Variación'], row['Nombre'], supermercado_id, categoria_id))

# Confirmar los cambios en la tabla de datos
conexionfacua.commit()

print("Datos insertados correctamente.")


Datos insertados correctamente.
